In [1]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(16):
    # Directory containing your event files
    log_dir = f"/home/<User>/repos/csg_is/calculating_scores/CFL/16Nodes_non_iid/fedstellar_CFL_17_07_2024_22_21_32/metrics/participant_{i}"
    date_str = "17/07/2024 20:21:33" # minus two hours for timezone

    date_obj = datetime.strptime(date_str.rstrip(" "), "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

    step      value                     timestamp                    tag
0  10815   4.300000 2024-07-17 21:03:49.006846208  Resources/CPU_percent
1  10825   2.800000 2024-07-17 21:03:59.032953344  Resources/CPU_percent
2  10835   2.900000 2024-07-17 21:04:09.110151936  Resources/CPU_percent
3  10845   4.000000 2024-07-17 21:04:19.137010432  Resources/CPU_percent
4  10855  44.099998 2024-07-17 21:04:29.319134464  Resources/CPU_percent
2024-07-17 21:14:19 2024-07-17 20:21:59
                 step     value                     timestamp
tag                                                          
Test/Accuracy   11435  0.358454 2024-07-17 21:14:13.818528256
Test/F1Score    11435  0.287346 2024-07-17 21:14:13.818560000
Test/Loss       11435  3.276053 2024-07-17 21:14:13.818499840
Test/Precision  11435  0.302195 2024-07-17 21:14:13.818540032
Test/Recall     11435  0.358454 2024-07-17 21:14:13.818550272
   step      value                     timestamp                    tag
0  1992  97.9000

In [2]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,11435,0.358454,2024-07-17 21:14:13.818528256,participant_0
Test/F1Score,11435,0.287346,2024-07-17 21:14:13.818560000,participant_0
Test/Loss,11435,3.276053,2024-07-17 21:14:13.818499840,participant_0
Test/Precision,11435,0.302195,2024-07-17 21:14:13.818540032,participant_0
Test/Recall,11435,0.358454,2024-07-17 21:14:13.818550272,participant_0
...,...,...,...,...
Test/Accuracy,14819,0.931426,2024-07-17 21:25:54.153306368,participant_15
Test/F1Score,14819,0.921501,2024-07-17 21:25:54.153337600,participant_15
Test/Loss,14819,0.169799,2024-07-17 21:25:54.153275392,participant_15


In [3]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_32834/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,13108.25,0.9242
Test/F1Score,13108.25,0.9119
Test/Loss,13108.25,0.2778
Test/Precision,13108.25,0.9141
Test/Recall,13108.25,0.9242


In [4]:
concat_df.groupby("tag").std().round(2)

/tmp/ipykernel_32834/582865284.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").std().round(2)


,step,value
tag,,
Test/Accuracy,1572.98,0.16
Test/F1Score,1572.98,0.17
Test/Loss,1572.98,0.80
Test/Precision,1572.98,0.17
Test/Recall,1572.98,0.16


In [ ]:
test_results = concat_df.groupby("tag").mean().round(4)
test_results[test_results.index.str.startswith("Test/")]